In [20]:
import math
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import gzip
import os
import time

In [21]:
#loading data set
DATA_PATH = "data_set.csv.gz"
#take top 20 currencies
CURRENCIES = {"USD", "EUR", "JPY", "GBP", "CNH", "AUD", "CAD", "CHF", "HKD", "SGD", "SEK", "KRW", "NZD", "NOK", "MXN", "INR"}

In [22]:
df = pd.read_csv(DATA_PATH, compression="gzip")
df

,ticker,participant_timestamp,open,high,low,close,volume
0,C:AUDCAD,1735775999999000000,0.890400,0.891829,0.888030,0.890250,5708
1,C:CADJPY,1735775999999000000,109.290000,109.747000,108.620000,109.704000,5859
2,C:USDEGP,1735775999999000000,50.818484,50.825360,50.748400,50.801600,392
3,C:BRLARS,1735775999999000000,166.691200,166.878230,163.144698,163.550800,580
4,C:CNHUSD,1735775999999000000,0.136323,0.136396,0.136110,0.136362,3176
...,...,...,...,...,...,...,...
79836,C:CADVND,1743379199999000000,17848.161395,17848.161395,17848.161395,17848.161395,1
79837,C:NZDINR,1743379199999000000,48.834077,48.834077,48.834077,48.834077,1
79838,C:JPYZAR,1743379199999000000,0.122656,0.123411,0.122555,0.123109,4042
79839,C:DKKSEK,1743379199999000000,1.444860,1.452780,1.444860,1.451770,6161


In [23]:
# Clean, filter, and sort the raw dataframe.

def is_symbol_of_interest(symbol: str):
    # First three characters after 'C:'
    currency1 = symbol[2:5]  
    currency2 = symbol[5:8]
    return currency1 in CURRENCIES and currency2 in CURRENCIES

# Sort by the timestamp and convert the timestamp to seconds.
# The timestamp is provided in nanoseconds but has 1 second resolution.
# df = df.sort_values(by="participant_timestamp", kind="stable")
# df["participant_timestamp"] //= int(1e9)

# The exchange always seems to equal 48.

# Remove currencies in which we are not interested.
# df = df[df["ticker"].map(is_symbol_of_interest)]

# OPTIONAL
# Drop duplicate symbols within the same timestamp.
# When this happens, we should keep the last (most up-to-date) prices.
# df = df.drop_duplicates(subset=["ticker", "participant_timestamp"], keep="last")
df.to_csv("filtered_data_set.csv")
df

,ticker,participant_timestamp,open,high,low,close,volume
0,C:AUDCAD,1735775999999000000,0.890400,0.891829,0.888030,0.890250,5708
1,C:CADJPY,1735775999999000000,109.290000,109.747000,108.620000,109.704000,5859
2,C:USDEGP,1735775999999000000,50.818484,50.825360,50.748400,50.801600,392
3,C:BRLARS,1735775999999000000,166.691200,166.878230,163.144698,163.550800,580
4,C:CNHUSD,1735775999999000000,0.136323,0.136396,0.136110,0.136362,3176
...,...,...,...,...,...,...,...
79836,C:CADVND,1743379199999000000,17848.161395,17848.161395,17848.161395,17848.161395,1
79837,C:NZDINR,1743379199999000000,48.834077,48.834077,48.834077,48.834077,1
79838,C:JPYZAR,1743379199999000000,0.122656,0.123411,0.122555,0.123109,4042
79839,C:DKKSEK,1743379199999000000,1.444860,1.452780,1.444860,1.451770,6161


In [24]:
# Add this before filtering to see what's happening
print(f"Original dataset size: {len(df)}")
print(f"Unique tickers in original data: {df['ticker'].nunique()}")
print(f"Sample tickers: {df['ticker'].head(10).tolist()}")

# Check how many match your criteria
matching_tickers = df[df["ticker"].map(is_symbol_of_interest)]
print(f"Rows with currencies of interest: {len(matching_tickers)}")
print(f"Unique matching tickers: {matching_tickers['ticker'].nunique()}")

# Check timestamp distribution
print(f"Unique timestamps: {df['participant_timestamp'].nunique()}")

Original dataset size: 79841
Unique tickers in original data: 1233
Sample tickers: ['C:AUDCAD', 'C:CADJPY', 'C:USDEGP', 'C:BRLARS', 'C:CNHUSD', 'C:USDCNH', 'C:GBPEGP', 'C:NZDCAD', 'C:CHFHKD', 'C:ILSAUD']
Rows with currencies of interest: 12792
Unique matching tickers: 194
Unique timestamps: 66
